In [1]:
"""
基于 Graph API 的并行化工作流实现
"""

from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="Qwen/Qwen2.5-7B-Instruct")


class State(TypedDict):
    topic: str
    joke: str
    story: str
    poem: str
    combined_output: str


def call_llm_1(state: State):
    msg = llm.invoke(f"写一个关于{state['topic']}的笑话")
    return {"joke": msg.content}


def call_llm_2(state: State):
    msg = llm.invoke(f"写一个关于{state['topic']}的故事")
    return {"story": msg.content}


def call_llm_3(state: State):
    msg = llm.invoke(f"写一首关于{state['topic']}的诗歌")
    return {"poem": msg.content}


def aggregator(state: State):
    combined = f"这是一个关于{state['topic']}的故事、笑话和诗歌！\n\n"
    combined += f"故事：\n{state['story']}\n\n"
    combined += f"笑话：\n{state['joke']}\n\n"
    combined += f"诗歌：\n{state['poem']}"
    return {"combined_output": combined}


parallel_builder = StateGraph(State)

parallel_builder.add_node("call_llm_1", call_llm_1)
parallel_builder.add_node("call_llm_2", call_llm_2)
parallel_builder.add_node("call_llm_3", call_llm_3)
parallel_builder.add_node("aggregator", aggregator)


parallel_builder.add_edge(START, "call_llm_1")
parallel_builder.add_edge(START, "call_llm_2")
parallel_builder.add_edge(START, "call_llm_3")
parallel_builder.add_edge("call_llm_1", "aggregator")
parallel_builder.add_edge("call_llm_2", "aggregator")
parallel_builder.add_edge("call_llm_3", "aggregator")

parallel_builder.add_edge("aggregator", END)

parallel_workflow = parallel_builder.compile()

state = parallel_workflow.invoke({"topic": "cats"})
print(state["combined_output"])

这是一个关于cats的故事、笑话和诗歌！

故事：
在一个平静祥和的小镇里，有一只名叫小云的猫咪。小云有着一身柔软的灰白相间的毛发，一双水汪汪的大眼睛总是好奇地打量着周围的一切。她有着一个特别的梦想：成为一名最受小镇居民欢迎的宠物医生。

小云的家在一个温馨的小公寓里，这里不仅住着她，还收养了小镇上许多需要帮助的流浪猫。她的主人鼓励她去实现梦想，并带她去报名参加了小镇的宠物护理课程。通过不懈的努力和学习，小云终于获得了成为一名宠物医生的资格。

但是，成为一名合格的宠物医生不可能一蹴而就。小云每天都在经历着各种挑战，比如学习如何为不同大小和品种的动物做治疗，如何读懂每一种病症的症状，如何安抚心情紧张的动物们，以及如何面对那些因为无法治愈疾病而放弃希望的主人……

有一天，小镇突然出现了一种奇怪的病症：许多家中的宠物开始出现异样的行为，一些犬只变得格外凶猛，而一些猫则失去了对它们最喜爱的事物的兴趣。这令小镇上的宠物主人和动物医院的工作人员们感到十分焦急。大家都不知道这种神秘疾病是什么，也不清楚该如何治疗。

小云知道了这个消息后，立刻决定要帮助那些生病的动物们。她在空闲时间加班加点查阅书籍，搜集信息，和主人交流，甚至用她这段时间学到的技能来照顾生病的宠物们。经过无数次的尝试和试验，终于有一天，小云发现所有的病猫有一个共同的症状：黏液堵塞在喉咙里，导致它们无法正常呼吸，食欲下降，表现得情绪低落。她立即着手研制了一种能够帮助清洁喉咙黏液的溶液。

这份鲜为人知的治疗方法，最终得到了广泛的认可和使用。小云通过解决这个难题不仅让许多动物恢复了健康，还在小镇上赢得了更多的尊重和认可。

随着时间的流逝，小云的宠物医院逐渐成为小镇上最著名的宠物医院，并且在她的带领下，也帮助更多的动物重返健康。她用爱心、耐心和专业技能，不仅治愈了小动物的疾病，还治愈了人们的心。

小云的故事向我们展示了爱、勇气和坚持的力量。无论遇到何种困难，只要心中有爱，就有无穷的勇气去克服它，去改变周围的世界。

笑话：
当然可以！这里有一个关于猫的笑话，希望能让你会心一笑：

为什么猫咪不喜欢上网？

因为他们总是被“猫被偷走了”的恐怖消息吓到！

诗歌：
当然可以，下面是一首关于猫咪们的诗歌：

猫语轻启夜的扉，
月牙形的瞳，梦的扉。
轻移步，无声响，
在银色月光下，
悄悄弹跳过影。

毛色斑斓如彩虹，
或